# Workshop - $\textit{Hands On}$!

In [ ]:
#Comando para instalar tensorflow
!pip install tensorflow

In [ ]:
#Comando para instalar tensorflow com suporte a GPU (apenas rexecute esta célula se tiver uma GPU com CUDA e cuDNN!!!!)
!pip install tensorflow-gpu

In [ ]:
# Comando para instalar os pacotes necessários
!pip install numpy pandas matplotlib seaborn sklearn xgboost keras

In [ ]:
import tensorflow
tensorflow.logging.set_verbosity(tensorflow.logging.ERROR)

In [ ]:
# Rodar apenas se estiver rodando GPU localmente e tiver problemas com alocação de memória!!
config = tensorflow.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
from keras.backend.tensorflow_backend import set_session
set_session(tensorflow.Session(config=config))

In [ ]:
from keras import models
from keras import layers
from keras import optimizers

In [ ]:
import numpy
import pandas
import xgboost
import seaborn
from sklearn import svm
from sklearn import tree
from sklearn import metrics
from sklearn import ensemble
from sklearn import neighbors
from matplotlib import pyplot
from sklearn import naive_bayes
from sklearn import linear_model
from sklearn import model_selection

In [ ]:
seaborn.set_style("darkgrid")
models_list = {}
auc_list = {}

# Introdução

<p>Neste <i>workshop</i> serão feitas várias demostrações de modelos de aprendizado para classificação supervisionada, utilizando algoritmos de aprendizado linear, estatístico, diferentes tipos de <i>ensembles</i> e ao final um exemplo com uma rede neural construída com o <i>Keras</i>.</p>

# Leitura e Análise Dos Dados

A nossa base de dados é um compilado de informações sobre exoplanetas da NASA (proveniente do [NASA Exoplanet Archive](https://exoplanetarchive.ipac.caltech.edu/index.html)) que possui várias informações sobre planetas confirmados exoplanetas e falsos positivos.

In [ ]:
df = pandas.read_csv("koi_database.csv")
classes = ["Confirmados", "Falsos Positivos"]
df.head()

In [ ]:
print("Linhas:", df.shape[0])
print("Colunas:", df.shape[1])

Aqui vamos eliminar colunas desnecessárias para o aprendizado, e também colunas que enviesam os algoritmos (informações disponíveis após a confirmação).

In [ ]:
df = df.drop(["kepoi_name", "kepid", "kepler_name", "koi_fpflag_nt", "koi_score", "koi_pdisposition", "koi_tce_delivname", "koi_teq_err1", "koi_teq_err2", "koi_fpflag_ss", "koi_fpflag_co", "koi_fpflag_ec"], axis=1)
df = df[df["koi_disposition"] != "CANDIDATE"]
df = df.dropna()
df.head()

Aqui aplicamos <i>one-hot encoding</i> na base para determinarmos as classes 0 e 1.

In [ ]:
df["koi_disposition"] = df["koi_disposition"].apply(lambda x: 1 if x == "CONFIRMED" else 0)
df.head()

In [ ]:
y = df["koi_disposition"]
pyplot.subplots(figsize=(8, 10))
seaborn.barplot(data=pandas.DataFrame({"Confirmados": y.sum(), "Falsos Positivos": y.size - y.sum()}, index=classes)).set_title("Classes", fontsize=20)
pyplot.show()
print("Confirmados:", y.sum())
print("Falsos Positivos:", y.size - y.sum())

Aui é feita a normalização <i>z-score</i> em nossos dados para ajudar na perda dos modelos.

In [ ]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.33, random_state=42)
mi = numpy.mean(x_train)
sigma = numpy.std(x_train)
x_train = (x_train - mi) / sigma
x_test = (x_test - mi) / sigma

In [ ]:
for i in df.columns[1:]:
    _, axis = pyplot.subplots(figsize=(20, 8))
    seaborn.distplot(df[i][df["koi_disposition"] == 1], bins=256, label="Confirmados")
    seaborn.distplot(df[i][df["koi_disposition"] == 0], bins=256, label="Falsos Positivos")
    axis.set_title("Histograma De Feature: " + i,fontsize=20)
    pyplot.legend(fontsize=15)
    pyplot.show()
    print(pandas.DataFrame({"Média": [numpy.mean(df[i][df["koi_disposition"] == 1]), numpy.mean(df[i][df["koi_disposition"] == 0])],
                            "Variância": [numpy.var(df[i][df["koi_disposition"] == 1]), numpy.var(df[i][df["koi_disposition"] == 0])]}, index=classes))

In [ ]:
pyplot.subplots(figsize=(8, 10))
seaborn.barplot(data=pandas.DataFrame({"Confirmados": y_train.sum(), "Falsos Positivos": y_train.size - y_train.sum()}, index=classes)).set_title("Classes (Treino e Validação)")
pyplot.show()
print("Confirmados:", y_train.sum())
print("Falsos Positivos:", y_train.size - y_train.sum())

In [ ]:
pyplot.subplots(figsize=(8, 10))
seaborn.barplot(data=pandas.DataFrame({"Confirmados": y_test.sum(), "Falsos Positivos": y_test.size - y_test.sum()}, index=classes)).set_title("Classes (Teste)")
pyplot.show()
print("Confirmados:", y_test.sum())
print("Falsos Positivos:", y_test.size - y_test.sum())

# <i>K-Fold</i>

In [ ]:
sets = list(model_selection.KFold(5, shuffle=True).split(x_train))

In [ ]:
def cross_validation(model, sets, x, y, p):
    l1 = []
    l2 = []
    for train_set, test_set in sets:
        x_train, y_train = x.iloc[train_set], y.iloc[train_set]
        x_test, y_test = x.iloc[test_set], y.iloc[test_set]
        l = model(x_train, y_train, x_test, y_test, p)
        l1.append(l[0])
        l2.append(l[1])
    return l1, l2

# Perceptron

In [ ]:
def linear_perceptron(x_train, y_train, x_test, y_test, p):
    pt = linear_model.Perceptron(penalty=p)
    pt.fit(x_train, y_train)
    return (metrics.accuracy_score(y_train, pt.predict(x_train)), metrics.accuracy_score(y_test, pt.predict(x_test)))

In [ ]:
pt1_mean, pt1_var, pt2_mean, pt2_var = [], [], [], []
for i in ["l2", "l1", "elasticnet"]:
    a1, a2 = cross_validation(linear_perceptron, sets, x_train, y_train, i)
    pt1_mean.append(numpy.mean(a1))
    pt1_var.append(numpy.var(a1))
    pt2_mean.append(numpy.mean(a2))
    pt2_var.append(numpy.var(a2))

In [ ]:
pandas.DataFrame({"Treino (Média)": pt1_mean, "Treino (Variância)": pt1_var, "Validação (Média)": pt2_mean, "Validação (Variância)": pt2_var}, index=["l2", "l1", "elasticnet"])

In [ ]:
# Insira aqui o seu código selecionando o hiperparâmetro ótimo!


In [ ]:
pt_y_pred = NOME_DO_SEU_MODELO_AQUI.predict(x_test)
ac = metrics.accuracy_score(y_test, pt_y_pred)
models_list["Perceptron"] = ac
print("Accurácia:", ac)

In [ ]:
cm = metrics.confusion_matrix(y_test, pt_y_pred)
pyplot.figure(figsize=(10, 10))
seaborn.set(font_scale=1.2)
seaborn.heatmap(pandas.DataFrame(cm, index=classes, columns=classes), annot=False, cmap="BuPu").set_title("Matriz De Confusão")
pyplot.xlabel("Predito", fontsize=18)
pyplot.ylabel("Real", fontsize=18)
pyplot.show()

In [ ]:
fpr, tpr, threshold = metrics.roc_curve(y_test, pt_y_pred)
pyplot.subplots(figsize=(12, 10))
pyplot.plot([0,1], [0, 1], "k--")
pyplot.plot(fpr, tpr)
pyplot.xlabel("Recall")
pyplot.ylabel("Precision")
pyplot.title("ROC Curve")
pyplot.show()
auc = metrics.auc(fpr, tpr)
auc_list["Perceptron"] = auc
print("AUC Score:", auc)

# <i>Naive Bayes</i>

In [ ]:
def gaussian_naive_bayes(x_train, y_train, x_test, y_test, p):
    gnv = naive_bayes.GaussianNB()
    gnv.fit(x_train, y_train)
    return (metrics.accuracy_score(y_train, gnv.predict(x_train)), metrics.accuracy_score(y_test, gnv.predict(x_test)))

In [ ]:
g1, g2 = cross_validation(gaussian_naive_bayes, sets, x_train, y_train, None)
pandas.DataFrame({"Média": [numpy.mean(g1), numpy.mean(g2)], "Variância": [numpy.var(g1), numpy.var(g2)]}, index=["Treino", "Validação"])

In [ ]:
# Insira aqui o seu código selecionando o hiperparâmetro ótimo!


In [ ]:
gnv_y_pred = NOME_DO_SEU_MODELO_AQUI.predict(x_test)
gnv_y_pred_proba = NOME_DO_SEU_MODELO_AQUI.predict_proba(x_test)[:, 1]
ac = metrics.accuracy_score(y_test, gnv_y_pred)
models_list["Naive Bayes"] = ac
print("Acurácia:", ac)

In [ ]:
cm = metrics.confusion_matrix(y_test, gnv_y_pred)
pyplot.figure(figsize=(10, 10))
seaborn.set(font_scale=1.2)
seaborn.heatmap(pandas.DataFrame(cm, index=classes, columns=classes), annot=False, cmap="BuPu").set_title("Matriz De Confusão", fontsize=20)
pyplot.xlabel("Predito", fontsize=18)
pyplot.ylabel("Real", fontsize=18)
pyplot.show()

In [ ]:
fpr, tpr, threshold = metrics.roc_curve(y_test, gnv_y_pred_proba)
pyplot.subplots(figsize=(12, 10))
pyplot.plot([0,1], [0, 1], "k--")
pyplot.plot(fpr, tpr)
pyplot.xlabel("Recall")
pyplot.ylabel("Precision")
pyplot.title("ROC Curve")
pyplot.show()
auc = metrics.auc(fpr, tpr)
auc_list["Naive Bayes"] = auc
print("AUC Score:", auc)

# Regressão Logística

In [ ]:
def logistic_regression(x_train, y_train, x_test, y_test, p):
    lr = linear_model.LogisticRegression(solver=p, max_iter=1000, n_jobs=4)
    lr.fit(x_train, y_train)
    return (metrics.accuracy_score(y_train, lr.predict(x_train)), metrics.accuracy_score(y_test, lr.predict(x_test)))

In [ ]:
lr1_mean, lr1_var, lr2_mean, lr2_var = [], [], [], []
for i in ["newton-cg", "lbfgs", "liblinear", "sag", "saga"]:
    a1, a2 = cross_validation(logistic_regression, sets, x_train, y_train, i)
    lr1_mean.append(numpy.mean(a1))
    lr1_var.append(numpy.var(a1))
    lr2_mean.append(numpy.mean(a2))
    lr2_var.append(numpy.var(a2))

In [ ]:
pandas.DataFrame({"Treino (Média)": lr1_mean, "Treino (Variância)": lr1_var, "Validação (Média)": lr2_mean, "Validação (Variância)": lr2_var}, index=["newton-cg", "lbfgs", "liblinear", "sag", "saga"])

In [ ]:
# Insira aqui o seu código selecionando o hiperparâmetro ótimo!


In [ ]:
lr_y_pred = NOME_DO_SEU_MODELO_AQUI.predict(x_test)
lr_y_pred_proba = NOME_DO_SEU_MODELO_AQUI.predict_proba(x_test)[:, 1]
ac = metrics.accuracy_score(y_test, lr_y_pred)
models_list["Regressão Logística"] = ac
print("Accurácia:", ac)

In [ ]:
cm = metrics.confusion_matrix(y_test, lr_y_pred)
pyplot.figure(figsize=(10, 10))
seaborn.set(font_scale=1.2)
seaborn.heatmap(pandas.DataFrame(cm, index=classes, columns=classes), annot=False, cmap="BuPu").set_title("Matriz De Confusão", fontsize=20)
pyplot.xlabel("Predito", fontsize=18)
pyplot.ylabel("Real", fontsize=18)
pyplot.show()

In [ ]:
fpr, tpr, threshold = metrics.roc_curve(y_test, lr_y_pred_proba)
pyplot.subplots(figsize=(12, 10))
pyplot.plot([0,1], [0, 1], "k--")
pyplot.plot(fpr, tpr)
pyplot.xlabel("Recall")
pyplot.ylabel("Precision")
pyplot.title("ROC Curve")
pyplot.show()
auc = metrics.auc(fpr, tpr)
auc_list["Regressão Logística"] = auc
print("AUC Score:", auc)

# Árvore De Decisão

In [ ]:
def decision_tree(x_train, y_train, x_test, y_test, p):
    t = tree.DecisionTreeClassifier(max_depth=p)
    t.fit(x_train, y_train)
    return (metrics.accuracy_score(y_train, t.predict(x_train)), metrics.accuracy_score(y_test, t.predict(x_test)))

In [ ]:
t1_mean, t1_var, t2_mean, t2_var = [], [], [], []
for i in range(1, 31):
    a1, a2 = cross_validation(decision_tree, sets, x_train, y_train, i)
    t1_mean.append(numpy.mean(a1))
    t1_var.append(numpy.var(a1))
    t2_mean.append(numpy.mean(a2))
    t2_var.append(numpy.var(a2))

In [ ]:
pyplot.subplots(figsize=(14, 8))
pyplot.xlabel("Profundidade", fontsize=15)
pyplot.ylabel("Acurácia (Média)", fontsize=15)
pyplot.plot(numpy.arange(1, 31, 1), t1_mean, "r-", label="Treino")
pyplot.plot(numpy.arange(1, 31, 1), t2_mean, "b-", label="Validação")
pyplot.legend(fontsize=15)
pyplot.show()

In [ ]:
pyplot.subplots(figsize=(14, 8))
pyplot.xlabel("Profundidade", fontsize=15)
pyplot.ylabel("Variância (Média)", fontsize=15)
pyplot.plot(numpy.arange(1, 31, 1), t1_var, "r-", label="Treino")
pyplot.plot(numpy.arange(1, 31, 1), t2_var, "b-", label="Validação")
pyplot.legend(fontsize=15)
pyplot.show()

In [ ]:
# Insira aqui o seu código selecionando o hiperparâmetro ótimo!


In [ ]:
t_y_pred = NOME_DO_SEU_MODELO_AQUI.predict(x_test)
t_y_pred_proba = NOME_DO_SEU_MODELO_AQUI.predict_proba(x_test)[:, 1]
ac = metrics.accuracy_score(y_test, t_y_pred)
models_list["Árvove De Decisão"] = ac
print("Acurácia:", ac)

In [ ]:
cm = metrics.confusion_matrix(y_test, t_y_pred)
pyplot.figure(figsize=(10, 10))
seaborn.set(font_scale=1.2)
seaborn.heatmap(pandas.DataFrame(cm, index=classes, columns=classes), annot=False, cmap="BuPu").set_title("Matriz De Confusão", fontsize=20)
pyplot.xlabel("Predito", fontsize=18)
pyplot.ylabel("Real", fontsize=18)
pyplot.show()

In [ ]:
fpr, tpr, threshold = metrics.roc_curve(y_test, t_y_pred_proba)
pyplot.subplots(figsize=(12, 10))
pyplot.plot([0,1], [0, 1], "k--")
pyplot.plot(fpr, tpr)
pyplot.xlabel("Recall")
pyplot.ylabel("Precision")
pyplot.title("ROC Curve")
pyplot.show()
auc = metrics.auc(fpr, tpr)
auc_list["Árvore De Decisão"] = auc
print("AUC Score:", auc)

# <i>Support Vector Machine</i>

In [ ]:
def support_vector_machines(x_train, y_train, x_test, y_test, p):
    svm_ = svm.SVC(gamma="scale", kernel=p)
    svm_.fit(x_train, y_train)
    return (metrics.accuracy_score(y_train, svm_.predict(x_train)), metrics.accuracy_score(y_test, svm_.predict(x_test)))

In [ ]:
s1_mean, s1_var, s2_mean, s2_var = [], [], [], []
for i in ["linear", "poly", "rbf", "sigmoid"]:
    a1, a2 = cross_validation(support_vector_machines, sets, x_train, y_train, i)
    s1_mean.append(numpy.mean(a1))
    s1_var.append(numpy.var(a1))
    s2_mean.append(numpy.mean(a2))
    s2_var.append(numpy.var(a2))

In [ ]:
pandas.DataFrame({"Treino (Média)": s1_mean, "Treino (Variância)": s1_var, "Validação (Média)": s2_mean, "Validação (Variância)": s2_var}, index=["Linear", "Sigmóide", "Polinomial", "RBF"])

In [ ]:
# Insira aqui o seu código selecionando o hiperparâmetro ótimo! ATENÇÃO: quando criar o modelo do SVM, você precisa usar o parâmetro "probability=True" se não, não vai funcionar!!!


In [ ]:
svm_y_pred = NOME_DO_SEU_MODELO_AQUI.predict(x_test)
svm_y_pred_proba = NOME_DO_SEU_MODELO_AQUI.predict_proba(x_test)[:, 1]
ac = metrics.accuracy_score(y_test, svm_y_pred)
models_list["SVM"] = ac
print("Acurácia:", ac)

In [ ]:
cm = metrics.confusion_matrix(y_test, svm_y_pred)
pyplot.figure(figsize=(10, 10))
seaborn.set(font_scale=1.2)
seaborn.heatmap(pandas.DataFrame(cm, index=classes, columns=classes), annot=False, cmap="BuPu").set_title("Matriz De Confusão", fontsize=20)
pyplot.xlabel("Predito", fontsize=18)
pyplot.ylabel("Real", fontsize=18)
pyplot.show()

In [ ]:
fpr, tpr, threshold = metrics.roc_curve(y_test, )
pyplot.subplots(figsize=(12, 10))
pyplot.plot([0,1], [0, 1], "k--")
pyplot.plot(fpr, tpr)
pyplot.xlabel("Recall")
pyplot.ylabel("Precision")
pyplot.title("ROC Curve")
pyplot.show()
auc = metrics.auc(fpr, tpr)
auc_list["SVM"] = auc
print("AUC Score:", auc)

# <i>k-Nearest Neighbours</i>

In [ ]:
def k_nearest_neighbours(x_train, y_train, x_test, y_test, p):
    knn = neighbors.KNeighborsClassifier(n_neighbors=p, n_jobs=4)
    knn.fit(x_train, y_train)
    return (metrics.accuracy_score(y_train, knn.predict(x_train)), metrics.accuracy_score(y_test, knn.predict(x_test)))

In [ ]:
k1_mean, k1_var, k2_mean, k2_var = [], [], [], []
for i in range(1, 51):
    a1, a2 = cross_validation(k_nearest_neighbours, sets, x_train, y_train, i)
    k1_mean.append(numpy.mean(a1))
    k1_var.append(numpy.var(a1))
    k2_mean.append(numpy.mean(a2))
    k2_var.append(numpy.var(a2))

In [ ]:
pyplot.subplots(figsize=(12, 8))
pyplot.xlabel("Número De Vizinhos", fontsize=15)
pyplot.ylabel("Acurácia (Média)", fontsize=15)
pyplot.plot(numpy.arange(1, 51, 1), k1_mean, "r-", label="Treino")
pyplot.plot(numpy.arange(1, 51, 1), k2_mean, "b-", label="Validação")
pyplot.legend(fontsize=15)
pyplot.show()

In [ ]:
pyplot.subplots(figsize=(12, 8))
pyplot.xlabel("Número De Vizinhos", fontsize=15)
pyplot.ylabel("Variância (Média)", fontsize=15)
pyplot.plot(numpy.arange(1, 51, 1), k1_var, "r-", label="Treino")
pyplot.plot(numpy.arange(1, 51, 1), k2_var, "b-", label="Validação")
pyplot.legend(fontsize=15)
pyplot.show()

In [ ]:
# Insira aqui o seu código selecionando o hiperparâmetro ótimo! 


In [ ]:
knn_y_pred = NOME_DO_SEU_MODELO_AQUI.predict(x_test)
knn_y_pred_proba = NOME_DO_SEU_MODELO_AQUI.predict_proba(x_test)[:, 1]
ac = metrics.accuracy_score(y_test, knn_y_pred)
models_list["k-NN"] = ac
print("Acurácia:", ac)

In [ ]:
cm = metrics.confusion_matrix(y_test, knn_y_pred)
pyplot.figure(figsize=(10, 10))
seaborn.set(font_scale=1.2)
seaborn.heatmap(pandas.DataFrame(cm, index=classes, columns=classes), annot=False, cmap="BuPu").set_title("Matriz De Confusão", fontsize=20)
pyplot.xlabel("Predito", fontsize=18)
pyplot.ylabel("Real", fontsize=18)
pyplot.show()

In [ ]:
fpr, tpr, threshold = metrics.roc_curve(y_test, knn_y_pred_proba)
pyplot.subplots(figsize=(12, 10))
pyplot.plot([0,1], [0, 1], "k--")
pyplot.plot(fpr, tpr)
pyplot.xlabel("Recall")
pyplot.ylabel("Precision")
pyplot.title("ROC Curve")
pyplot.show()
auc = metrics.auc(fpr, tpr)
auc_list["k-NN"] = auc
print("AUC Score:", auc)

# <i>Adaboost</i>

In [ ]:
def adaboost(x_train, y_train, x_test, y_test, p):
    ab = ensemble.AdaBoostClassifier(n_estimators=p)
    ab.fit(x_train, y_train)
    return (metrics.accuracy_score(y_train, ab.predict(x_train)), metrics.accuracy_score(y_test, ab.predict(x_test)))

In [ ]:
ab1_mean, ab1_var, ab2_mean, ab2_var = [], [], [], []
for i in range(1, 51):
    a1, a2 = cross_validation(adaboost, sets, x_train, y_train, i)
    ab1_mean.append(numpy.mean(a1))
    ab1_var.append(numpy.var(a1))
    ab2_mean.append(numpy.mean(a2))
    ab2_var.append(numpy.var(a2))

In [ ]:
pyplot.subplots(figsize=(12, 8))
pyplot.xlabel("Número De Estimadores", fontsize=15)
pyplot.ylabel("Acurácia (Média)", fontsize=15)
pyplot.plot(numpy.arange(1, 51, 1), ab1_mean, "r-", label="Treino")
pyplot.plot(numpy.arange(1, 51, 1), ab2_mean, "b-", label="Validação")
pyplot.legend(fontsize=15)
pyplot.show()

In [ ]:
pyplot.subplots(figsize=(12, 8))
pyplot.xlabel("Número De Estimadores", fontsize=15)
pyplot.ylabel("Variância (Média)", fontsize=15)
pyplot.plot(numpy.arange(1, 51, 1), ab1_var, "r-", label="Treino")
pyplot.plot(numpy.arange(1, 51, 1), ab2_var, "b-", label="Validação")
pyplot.legend(fontsize=15)
pyplot.show()

In [ ]:
# Insira aqui o seu código selecionando o hiperparâmetro ótimo! 


In [ ]:
ab_y_pred = NOME_DO_SEU_MODELO_AQUI.predict(x_test)
ab_y_pred_proba = NOME_DO_SEU_MODELO_AQUI.predict_proba(x_test)
ac = metrics.accuracy_score(y_test, ab_y_pred)
models_list["Adaboost"] = ac
print("Acurácia:", ac)

In [ ]:
cm = metrics.confusion_matrix(y_test, ab_y_pred)
pyplot.figure(figsize=(10, 10))
seaborn.set(font_scale=1.2)
seaborn.heatmap(pandas.DataFrame(cm, index=classes, columns=classes), annot=False, cmap="BuPu").set_title("Matriz De Confusão", fontsize=20)
pyplot.xlabel("Predito", fontsize=18)
pyplot.ylabel("Real", fontsize=18)
pyplot.show()

In [ ]:
fpr, tpr, threshold = metrics.roc_curve(y_test, ab_y_pred_proba)
pyplot.subplots(figsize=(12, 10))
pyplot.plot([0,1], [0, 1], "k--")
pyplot.plot(fpr, tpr)
pyplot.xlabel("Recall")
pyplot.ylabel("Precision")
pyplot.title("ROC Curve")
pyplot.show()
auc = metrics.auc(fpr, tpr)
auc_list["Adaboost"] = auc
print("AUC Score:", auc)

# <i>Bagging</i>

In [ ]:
def bagging(x_train, y_train, x_test, y_test, p):
    bgg = ensemble.BaggingClassifier(n_jobs=4, n_estimators=p)
    bgg.fit(x_train, y_train)
    return (metrics.accuracy_score(y_train, bgg.predict(x_train)), metrics.accuracy_score(y_test, bgg.predict(x_test)))

In [ ]:
bgg1_mean, bgg1_var, bgg2_mean, bgg2_var = [], [], [], []
for i in range(1, 51):
    a1, a2 = cross_validation(bagging, sets, x_train, y_train, i)
    bgg1_mean.append(numpy.mean(a1))
    bgg1_var.append(numpy.var(a1))
    bgg2_mean.append(numpy.mean(a2))
    bgg2_var.append(numpy.var(a2))

In [ ]:
pyplot.subplots(figsize=(12, 8))
pyplot.xlabel("Número De Estimadores", fontsize=15)
pyplot.ylabel("Acurácia (Média)", fontsize=15)
pyplot.plot(numpy.arange(1, 51, 1), bgg1_mean, "r-", label="Treino")
pyplot.plot(numpy.arange(1, 51, 1), bgg2_mean, "b-", label="Validação")
pyplot.legend(fontsize=15)
pyplot.show()

In [ ]:
pyplot.subplots(figsize=(12, 8))
pyplot.xlabel("Número De Estimadores", fontsize=15)
pyplot.ylabel("Variância (Média)", fontsize=15)
pyplot.plot(numpy.arange(1, 51, 1), bgg1_var, "r-", label="Treino")
pyplot.plot(numpy.arange(1, 51, 1), bgg2_var, "b-", label="Validação")
pyplot.legend(fontsize=15)
pyplot.show()

In [ ]:
# Insira aqui o seu código selecionando o hiperparâmetro ótimo!

In [ ]:
bgg_y_pred = NOME_DO_SEU_MODELO_AQUI.predict(x_test)
bgg_y_pred_proba = NOME_DO_SEU_MODELO_AQUI.predict_proba(x_test)[:, 1]
ac = metrics.accuracy_score(y_test, bgg_y_pred)
models_list["Bagging"] = ac
print("Acurácia:", ac)

In [ ]:
cm = metrics.confusion_matrix(y_test, bgg_y_pred)
pyplot.figure(figsize=(10, 10))
seaborn.set(font_scale=1.2)
seaborn.heatmap(pandas.DataFrame(cm, index=classes, columns=classes), annot=False, cmap="BuPu").set_title("Matriz De Confusão", fontsize=20)
pyplot.xlabel("Predito", fontsize=18)
pyplot.ylabel("Real", fontsize=18)
pyplot.show()

In [ ]:
fpr, tpr, threshold = metrics.roc_curve(y_test, bgg_y_pred_proba)
pyplot.subplots(figsize=(12, 10))
pyplot.plot([0,1], [0, 1], "k--")
pyplot.plot(fpr, tpr)
pyplot.xlabel("Recall")
pyplot.ylabel("Precision")
pyplot.title("ROC Curve")
pyplot.show()
print("AUC Score:", metrics.auc(fpr, tpr))

# <i>Random Forest</i>

In [ ]:
def random_forest(x_train, y_train, x_test, y_test, p):
    rf = ensemble.RandomForestClassifier(max_depth=5, n_estimators=p)
    rf.fit(x_train, y_train)
    return (metrics.accuracy_score(y_train, rf.predict(x_train)), metrics.accuracy_score(y_test, rf.predict(x_test)))

In [ ]:
rf1_mean, rf1_var, rf2_mean, rf2_var = [], [], [], []
for i in range(1, 51):
    a1, a2 = cross_validation(random_forest, sets, x_train, y_train, i)
    rf1_mean.append(numpy.mean(a1))
    rf1_var.append(numpy.var(a1))
    rf2_mean.append(numpy.mean(a2))
    rf2_var.append(numpy.var(a2))

In [ ]:
pyplot.subplots(figsize=(12, 8))
pyplot.xlabel("Número De Estimadores", fontsize=15)
pyplot.ylabel("Acurácia (Média)", fontsize=15)
pyplot.plot(numpy.arange(1, 51, 1), rf1_mean, "r-", label="Treino")
pyplot.plot(numpy.arange(1, 51, 1), rf2_mean, "b-", label="Validação")
pyplot.legend(fontsize=15)
pyplot.show()

In [ ]:
pyplot.subplots(figsize=(12, 8))
pyplot.xlabel("Número De Estimadores", fontsize=15)
pyplot.ylabel("Variância (Média)", fontsize=15)
pyplot.plot(numpy.arange(1, 51, 1), rf1_var, "r-", label="Treino")
pyplot.plot(numpy.arange(1, 51, 1), rf2_var, "b-", label="Validação")
pyplot.legend(fontsize=15)
pyplot.show()

In [ ]:
# Insira aqui o seu código selecionando o hiperparâmetro ótimo!


In [ ]:
rf_y_pred = NOME_DO_SEU_MODELO_AQUI.predict(x_test)
rf_y_pred_proba = NOME_DO_SEU_MODELO_AQUI.predict_proba(x_test)[:, 1]
ac = metrics.accuracy_score(y_test, rf_y_pred)
models_list["Random Forest"] = ac
print("Acurácia:", ac)

In [ ]:
cm = metrics.confusion_matrix(y_test, rf_y_pred)
pyplot.figure(figsize=(10, 10))
seaborn.set(font_scale=1.2)
seaborn.heatmap(pandas.DataFrame(cm, index=classes, columns=classes), annot=False, cmap="BuPu").set_title("Matriz De Confusão", fontsize=20)
pyplot.xlabel("Predito", fontsize=18)
pyplot.ylabel("Real", fontsize=18)
pyplot.show()

In [ ]:
fpr, tpr, threshold = metrics.roc_curve(y_test, rf_y_pred_proba)
pyplot.subplots(figsize=(12, 10))
pyplot.plot([0,1], [0, 1], "k--")
pyplot.plot(fpr, tpr)
pyplot.xlabel("Recall")
pyplot.ylabel("Precision")
pyplot.title("ROC Curve")
pyplot.show()
auc = metrics.auc(fpr, tpr)
print("AUC Score:", auc)

# <i>Gradient Tree Boosting</i>

In [ ]:
def gradient_tree_boosting(x_train, y_train, x_test, y_test, p):
    gtb = ensemble.GradientBoostingClassifier(n_estimators=p)
    gtb.fit(x_train, y_train)
    return (metrics.accuracy_score(y_train, gtb.predict(x_train)), metrics.accuracy_score(y_test, gtb.predict(x_test)))

In [ ]:
gtb1_mean, gtb1_var, gtb2_mean, gtb2_var = [], [], [], []
for i in range(1, 51):
    a1, a2 = cross_validation(gradient_tree_boosting, sets, x_train, y_train, i)
    gtb1_mean.append(numpy.mean(a1))
    gtb1_var.append(numpy.var(a1))
    gtb2_mean.append(numpy.mean(a2))
    gtb2_var.append(numpy.var(a2))

In [ ]:
pyplot.subplots(figsize=(12, 8))
pyplot.xlabel("Número De Estimadores", fontsize=15)
pyplot.ylabel("Acurácia (Média)", fontsize=15)
pyplot.plot(numpy.arange(1, 51, 1), gtb1_mean, "r-", label="Treino")
pyplot.plot(numpy.arange(1, 51, 1), gtb2_mean, "b-", label="Validação")
pyplot.legend(fontsize=15)
pyplot.show()

In [ ]:
pyplot.subplots(figsize=(12, 8))
pyplot.xlabel("Número De Estimadores", fontsize=15)
pyplot.ylabel("Variância (Média)", fontsize=15)
pyplot.plot(numpy.arange(1, 51, 1), gtb1_var, "r-", label="Treino")
pyplot.plot(numpy.arange(1, 51, 1), gtb2_var, "b-", label="Validação")
pyplot.legend(fontsize=15)
pyplot.show()

In [ ]:
# Insira aqui o seu código selecionando o hiperparâmetro ótimo!


In [ ]:
gtb_y_pred = NOME_DO_SEU_MODELO_AQUI.predict(x_test)
gbt_y_pred_proba = NOME_DO_SEU_MODELO_AQUI.predict_proba(x_test)[:, 1]
ac = metrics.accuracy_score(y_test, gtb_y_pred)
models_list["Gradient Tree Boosting"] = ac
print("Acurácia:", ac)

In [ ]:
cm = metrics.confusion_matrix(y_test, gtb_y_pred)
pyplot.figure(figsize=(10, 10))
seaborn.set(font_scale=1.2)
seaborn.heatmap(pandas.DataFrame(cm, index=classes, columns=classes), annot=False, cmap="BuPu").set_title("Matriz De Confusão", fontsize=20)
pyplot.xlabel("Predito", fontsize=18)
pyplot.ylabel("Real", fontsize=18)
pyplot.show()

In [ ]:
fpr, tpr, threshold = metrics.roc_curve(y_test, gbt_y_pred_proba)
pyplot.subplots(figsize=(12, 10))
pyplot.plot([0,1], [0, 1], "k--")
pyplot.plot(fpr, tpr)
pyplot.xlabel("Recall")
pyplot.ylabel("Precision")
pyplot.title("ROC Curve")
pyplot.show()
auc = metrics.auc(fpr, tpr)
auc_list["Gradient Tree Boosting"] = auc
print("AUC Score:", auc)

# <i>XGBoost</i>

In [ ]:
def xgb_tree(x_train, y_train, x_test, y_test, p):
    xgb = xgboost.sklearn.XGBClassifier(n_jobs=4, n_estimators=p)
    xgb.fit(x_train, y_train)
    return (metrics.accuracy_score(y_train, xgb.predict(x_train)), metrics.accuracy_score(y_test, xgb.predict(x_test)))

In [ ]:
xgb1_mean, xgb1_var, xgb2_mean, xgb2_var = [], [], [], []
for i in range(1, 51):
    a1, a2 = cross_validation(xgb_tree, sets, x_train, y_train, i)
    xgb1_mean.append(numpy.mean(a1))
    xgb1_var.append(numpy.var(a1))
    xgb2_mean.append(numpy.mean(a2))
    xgb2_var.append(numpy.var(a2))

In [ ]:
pyplot.subplots(figsize=(12, 8))
pyplot.xlabel("Número De Estimadores", fontsize=15)
pyplot.ylabel("Acurácia (Média)", fontsize=15)
pyplot.plot(numpy.arange(1, 51, 1), xgb1_mean, "r-", label="Treino")
pyplot.plot(numpy.arange(1, 51, 1), xgb2_mean, "b-", label="Validação")
pyplot.legend(fontsize=15)
pyplot.show()

In [ ]:
pyplot.subplots(figsize=(12, 8))
pyplot.xlabel("Número De Estimadores", fontsize=15)
pyplot.ylabel("Variância (Média)", fontsize=15)
pyplot.plot(numpy.arange(1, 51, 1), xgb1_var, "r-", label="Treino")
pyplot.plot(numpy.arange(1, 51, 1), xgb2_var, "b-", label="Validação")
pyplot.legend(fontsize=15)
pyplot.show()

In [ ]:
# Insira aqui o seu código selecionando o hiperparâmetro ótimo!


In [ ]:
xgb_y_pred = NOME_DO_SEU_MODELO_AQUI.predict(x_test)
xgb_y_pred_proba = NOME_DO_SEU_MODELO_AQUI.predict_proba(x_test)[:, 1]
ac = metrics.accuracy_score(y_test, xgb_y_pred)
models_list["XGBoost"] = ac
print("Acurácia:", ac)

In [ ]:
cm = metrics.confusion_matrix(y_test, xgb_y_pred)
pyplot.figure(figsize=(10, 10))
seaborn.set(font_scale=1.2)
seaborn.heatmap(pandas.DataFrame(cm, index=classes, columns=classes), annot=False, cmap="BuPu").set_title("Matriz De Confusão", fontsize=20)
pyplot.xlabel("Predito", fontsize=18)
pyplot.ylabel("Real", fontsize=18)
pyplot.show()

In [ ]:
fpr, tpr, threshold = metrics.roc_curve(y_test, xgb_y_pred_proba)
pyplot.subplots(figsize=(12, 10))
pyplot.plot([0,1], [0, 1], "k--")
pyplot.plot(fpr, tpr)
pyplot.xlabel("Recall")
pyplot.ylabel("Precision")
pyplot.title("ROC Curve")
pyplot.show()
auc = metrics.auc(fpr, tpr)
auc_list["XGBoost"] = auc
print("AUC Score:", auc)

# Rede Neural Multicamadas

In [ ]:
def create_neural_network():
    net = models.Sequential()
    #Construa sua própria rede, use sua criatividade! Funções de ativação: "relu", "softsign", "tanh", "sigmoid", "softmax", "elu", "selu", "softplus", "exponential", "linear"
    #DICA 1: Use camadas com tamanho de potências de 2!
    #DICA 2: funções de ativação como "relu", "linear" e "softsign" são mais baratas computacionalmente!
    
    net.add(layers.Dense(1, activation="sigmoid", name="Output"))
    return net

In [ ]:
def neural_network(x_train, y_train, x_test, y_test, p):
    net = create_neural_network()
    net.compile(optimizer=optimizers.Adagrad(lr=0.0253), loss="binary_crossentropy")
    net.fit(x_train.values, y_train.values, epochs=50, batch_size=32, verbose=0)
    return (metrics.accuracy_score(y_train, net.predict_classes(x_train.values)), metrics.accuracy_score(y_test, net.predict_classes(x_test.values)))

In [ ]:
a1, a2 = cross_validation(neural_network, sets, x_train, y_train, None)
pandas.DataFrame({"Média": [numpy.mean(a1), numpy.mean(a2)], "Variância": [numpy.var(a1), numpy.var(a2)]}, index=["Treino", "Validação"])

In [ ]:
net = create_neural_network()
net.compile(optimizer=optimizers.Adagrad(lr=0.0253), loss="binary_crossentropy")
net.fit(x_train.values, y_train.values, epochs=50, batch_size=32)

In [ ]:
net_y_pred = net.predict_classes(x_test.values)
ac = metrics.accuracy_score(y_test, net_y_pred)
models_list["Rede Neural Multicamadas"] = ac
print("Acurácia:", ac)

In [ ]:
cm = metrics.confusion_matrix(y_test, net_y_pred)
pyplot.figure(figsize=(10, 10))
seaborn.set(font_scale=1.2)
seaborn.heatmap(pandas.DataFrame(cm, index=classes, columns=classes), annot=False, cmap="BuPu").set_title("Matriz De Confusão", fontsize=20)
pyplot.xlabel("Predito", fontsize=18)
pyplot.ylabel("Real", fontsize=18)
pyplot.show()

In [ ]:
fpr, tpr, threshold = metrics.roc_curve(y_test, net.predict(x_test))
pyplot.subplots(figsize=(12, 10))
pyplot.plot([0,1], [0, 1], "k--")
pyplot.plot(fpr, tpr)
pyplot.xlabel("Recall")
pyplot.ylabel("Precision")
pyplot.title("ROC Curve")
pyplot.show()
auc = metrics.auc(fpr, tpr)
auc_list["Rede Neural Multicamadas"] = auc
print("AUC Score:", auc)

# Comparação Entre Modelos

In [ ]:
z = sorted(models_list.items(), key=lambda x: x[1])
zlist = list(zip(*z))
pyplot.subplots(figsize=(16, 8))
pyplot.barh(zlist[0], zlist[1])
pyplot.title("Acurácia (Teste)")
pyplot.show()

In [ ]:
z = sorted(auc_list.items(), key=lambda x: x[1])
zlist = list(zip(*z))
pyplot.subplots(figsize=(16, 8))
pyplot.barh(zlist[0], zlist[1])
pyplot.title("AUC (Teste)")
pyplot.show()